In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import qgrid
import pandas_profiling
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

C:\Users\Naresh\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.

C:\Users\Naresh\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [2]:
df_abilities = pd.read_excel(r"D:\Naresh\BG\Abilities.xlsx")
df_knowledge = pd.read_excel(r"D:\Naresh\BG\Knowledge.xlsx")
df_skills = pd.read_excel(r"D:\Naresh\BG\Skills.xlsx")

In [3]:
qgrid_widget = qgrid.QGridWidget(df=df_abilities,show_toolbar=True)
qgrid_widget

A Jupyter Widget

In [4]:
qgrid_widget = qgrid.QGridWidget(df=df_knowledge,show_toolbar=True)
qgrid_widget

A Jupyter Widget

In [5]:
qgrid_widget = qgrid.QGridWidget(df=df_skills,show_toolbar=True)
qgrid_widget

A Jupyter Widget

In [6]:
pandas_profiling.ProfileReport(df_abilities)

Number of variables,15
Number of observations,100568
Total Missing (%),3.4%
Total size in memory,11.5 MiB
Average record size in memory,120.0 B
Numeric,2
Categorical,10
Boolean,1
Date,0
Text (Unique),0
Rejected,2


##### Filtering Level values only from Abilities, Knowledge and Skills data set. As Level indicates capability required and I will be using that for predictions.

In [7]:
print(df_abilities.shape)
print(df_knowledge.shape)
print(df_skills.shape)

(100568, 15)
(63822, 15)
(67690, 15)


In [8]:
df_abilities=df_abilities[df_abilities['Scale ID']=='LV']
df_knowledge=df_knowledge[df_knowledge['Scale ID']=='LV']
df_skills=df_skills[df_skills['Scale ID']=='LV']

In [9]:
print(df_abilities.shape)
print(df_knowledge.shape)
print(df_skills.shape)

(50284, 15)
(31911, 15)
(33845, 15)


#### Removing Not relevant attributes and recommend suppress as 'Y' from 3 data sets

In [10]:
df_combined=df_abilities.append(df_knowledge).append(df_skills)

In [11]:
df_combined.shape

(116040, 15)

In [12]:
df_combined=df_combined[(df_combined['Recommend Suppress']=='N') & (df_combined['Not Relevant']=='N')]

In [13]:
qgrid_widget = qgrid.QGridWidget(df=df_combined,show_toolbar=True)
qgrid_widget

A Jupyter Widget

In [14]:
df_employment=df_combined.pivot_table(index='O*NET-SOC Code',columns='Element Name',values='Data Value')

In [15]:
df_employment.head()

Element Name,Active Learning,Active Listening,Administration and Management,Arm-Hand Steadiness,Auditory Attention,Biology,Building and Construction,Category Flexibility,Chemistry,Clerical,...,Time Sharing,Transportation,Troubleshooting,Trunk Strength,Visual Color Discrimination,Visualization,Wrist-Finger Speed,Writing,Written Comprehension,Written Expression
O*NET-SOC Code,,,,,,,,,,,,,,,,,,,,,
11-1011.00,4.75,4.88,6.23,NaN,2.12,NaN,2.00,4.12,0.95,3.50,...,2.88,1.91,NaN,NaN,1.62,3.88,NaN,4.38,4.62,4.62
11-1011.03,3.75,4.12,NaN,NaN,1.62,NaN,NaN,3.62,NaN,NaN,...,2.38,NaN,NaN,NaN,1.62,3.12,NaN,4.38,4.38,4.38
11-1021.00,3.62,4.00,5.21,1.5,2.00,0.9,2.62,3.00,1.74,3.70,...,2.88,2.31,1.38,2.12,2.25,2.50,0.62,3.88,4.00,4.00
11-2011.00,4.12,4.12,4.41,0.5,1.25,NaN,NaN,3.88,NaN,4.34,...,2.12,1.59,NaN,0.88,2.75,3.12,0.75,3.88,4.00,4.00
11-2021.00,4.12,4.12,4.61,NaN,1.62,NaN,NaN,3.62,NaN,4.10,...,2.62,1.53,NaN,0.88,2.88,3.00,0.75,3.88,4.12,4.12


#### Creating new column by taking first 6 digits of SOC code, so that we can combine similar occupation for e.g. Accountant and Auditors

In [16]:
df_employment['SOC_trim'] = df_employment.index.str[:7]

In [17]:
df_employment.shape

(966, 120)

In [18]:
df_employment = df_employment.groupby('SOC_trim').mean()

In [19]:
df_employment['SOC'] = df_employment.index

#### For few occupation while aggregating the data, I have decided to ignore the null values during aggregation

#### Importing label data for 70 occupation used by authors in paper

In [20]:
df_label = pd.read_excel(r'D:\Naresh\BG\Labeling_work.xlsx')

In [21]:
df_employment.head()

Element Name,Active Learning,Active Listening,Administration and Management,Arm-Hand Steadiness,Auditory Attention,Biology,Building and Construction,Category Flexibility,Chemistry,Clerical,...,Transportation,Troubleshooting,Trunk Strength,Visual Color Discrimination,Visualization,Wrist-Finger Speed,Writing,Written Comprehension,Written Expression,SOC
SOC_trim,,,,,,,,,,,,,,,,,,,,,
11-1011,4.25,4.50,6.23,NaN,1.87,NaN,2.00,3.87,0.95,3.50,...,1.91,NaN,NaN,1.62,3.50,NaN,4.38,4.50,4.50,11-1011
11-1021,3.62,4.00,5.21,1.5,2.00,0.9,2.62,3.00,1.74,3.70,...,2.31,1.38,2.12,2.25,2.50,0.62,3.88,4.00,4.00,11-1021
11-2011,4.12,4.12,4.41,0.5,1.25,NaN,NaN,3.88,NaN,4.34,...,1.59,NaN,0.88,2.75,3.12,0.75,3.88,4.00,4.00,11-2011
11-2021,4.12,4.12,4.61,NaN,1.62,NaN,NaN,3.62,NaN,4.10,...,1.53,NaN,0.88,2.88,3.00,0.75,3.88,4.12,4.12,11-2021
11-2022,3.88,4.00,NaN,0.5,1.88,NaN,NaN,3.62,NaN,NaN,...,NaN,NaN,1.38,1.62,2.50,0.50,4.00,4.00,4.00,11-2022


In [22]:
df_label[df_label['SOC'].isin (df_employment.SOC)]

,Sr No,SOC,computerisable,original_probability
0,1,29-1125,NaN,0.0028
1,2,49-1011,NaN,0.0030
2,3,11-9161,NaN,0.0030
3,4,21-1023,NaN,0.0031
4,5,29-1181,NaN,0.0033
5,6,29-1122,NaN,0.0035
6,7,29-2091,NaN,0.0035
7,8,21-1022,NaN,0.0035
8,9,29-1022,NaN,0.0036
9,10,33-1021,NaN,0.0036


#### I was able to match 684 SOC occupation code given in paper to O*NET database

#### List of 18 SOC Code which did not match ONET data SOC code are listed below

In [23]:
df_label[~df_label['SOC'].isin (df_employment.SOC)]

,Sr No,SOC,computerisable,original_probability
14,15,29-1060,0.0,0.0042
45,46,29-1111,0.0,0.0090
47,48,25-3999,NaN,0.0095
111,112,25-1000,NaN,0.0320
141,142,29-9799,0.0,0.0550
206,207,39-4831,NaN,0.2000
207,208,15-1179,NaN,0.2100
211,212,15-1799,NaN,0.2200
217,218,29-2037,NaN,0.2300
241,242,13-1078,NaN,0.3100


In [24]:
df_labeleddata = pd.merge(df_label,df_employment,on=['SOC'])

#### Checking mean value difference between variables

In [25]:
df_labeleddata.groupby('computerisable').mean()

,Sr No,original_probability,Active Learning,Active Listening,Administration and Management,Arm-Hand Steadiness,Auditory Attention,Biology,Building and Construction,Category Flexibility,...,Time Sharing,Transportation,Troubleshooting,Trunk Strength,Visual Color Discrimination,Visualization,Wrist-Finger Speed,Writing,Written Comprehension,Written Expression
computerisable,,,,,,,,,,,,,,,,,,,,,
0.0,182.233333,0.204820,3.507000,3.856111,3.419167,2.230625,2.077278,1.895357,1.925357,3.340333,...,2.701722,1.883182,1.284524,2.213750,2.224943,2.894389,1.143509,3.581611,3.927444,3.704500
1.0,527.675676,0.845946,2.780135,3.324459,2.498281,2.162121,2.012297,1.694000,2.223889,2.969730,...,2.398649,2.029038,1.579773,2.078966,2.069324,2.449054,1.463889,3.023514,3.439459,3.147432


In [26]:
df_train_predictors = df_labeleddata[(df_labeleddata['computerisable']==1) | (df_labeleddata['computerisable']==0)].iloc[:,4:]
df_train_target = df_labeleddata[(df_labeleddata['computerisable']==1) | (df_labeleddata['computerisable']==0)].iloc[:,2:3]

In [27]:
df_test_predictors = df_labeleddata[~(df_labeleddata['computerisable']==1) | (df_labeleddata['computerisable']==0)].iloc[:,4:]
df_test_target = df_labeleddata[~(df_labeleddata['computerisable']==1) | (df_labeleddata['computerisable']==0)].iloc[:,3:4]

In [28]:
df_train_predictors=df_train_predictors.fillna(0)
df_test_predictors=df_test_predictors.fillna(0)

In [29]:
df_train_predictors.shape

(67, 119)

#### To get idea of which parameter is contributing more to computerisation of job, I am using linear model to understand the impact on final outcome

In [43]:
X= df_train_predictors
Y= df_train_target
X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:         computerisable   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Thu, 31 Jan 2019   Prob (F-statistic):                nan
Time:                        03:10:06   Log-Likelihood:                 2192.9
No. Observations:                  67   AIC:                            -4252.
Df Residuals:                       0   BIC:                            -4104.
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

C:\Users\Naresh\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1392: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\Naresh\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1392: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Naresh\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1400: RuntimeWarning:

divide by zero encountered in double_scalars

C:\Users\Naresh\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1353: RuntimeWarning:

divide by zero encountered in double_scalars



In [30]:
pandas_profiling.ProfileReport(df_train_predictors)

Number of variables,120
Number of observations,67
Total Missing (%),0.0%
Total size in memory,62.9 KiB
Average record size in memory,961.2 B
Numeric,84
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,36


In [31]:
df_train_predictor_std = pd.DataFrame(StandardScaler().fit_transform(df_train_predictors),columns=df_train_predictors.columns)
df_test_predictor_std = pd.DataFrame(StandardScaler().fit_transform(df_test_predictors),columns=df_test_predictors.columns)

##### Here in profile report we can see that 36 features have high colinearity with other variables and hence we can drope these columns

In [32]:
pca = PCA(0.95)
pca.fit_transform(df_train_predictor_std)
print (pca.explained_variance_ratio_)

[0.359493   0.17499335 0.11294998 0.0533498  0.03437974 0.02688574
 0.0208426  0.01985768 0.01681967 0.01356318 0.01248381 0.01187564
 0.01081125 0.00957557 0.00895349 0.00817693 0.007612   0.00688137
 0.00634623 0.006148   0.0055662  0.00510623 0.00478471 0.00425333
 0.00380876 0.00379054 0.00360397]


#### We can attain 95% variability using 27 Principal components which is almost 77.5% reduction in number of variables

In [33]:
df_trainpca_variable=pd.DataFrame(pca.fit_transform(df_train_predictor_std))


#### Creating 27 Principal components for test data set (i.e. 617 SOC which are not labeled), I am not using 0.95 variability parameter as for Test data we could achieve that variability with lesser/higher amount of variables.

In [34]:
pca_test = PCA(n_components=27)
df_testpca_variable=pd.DataFrame(pca_test.fit_transform(df_test_predictor_std))

In [35]:
df_train_target.shape

(67, 1)

In [36]:
pd.to_numeric(df_train_target.computerisable)

17     0.0
31     0.0
35     0.0
40     0.0
58     0.0
66     0.0
67     0.0
80     0.0
85     0.0
109    0.0
110    0.0
116    0.0
128    0.0
153    0.0
157    0.0
163    0.0
166    0.0
169    0.0
170    0.0
203    0.0
224    0.0
230    0.0
238    0.0
239    0.0
251    1.0
260    0.0
264    1.0
270    0.0
308    1.0
315    0.0
      ... 
409    0.0
425    1.0
427    1.0
437    1.0
469    1.0
482    1.0
493    1.0
506    1.0
509    1.0
513    1.0
514    1.0
515    1.0
568    1.0
570    1.0
571    1.0
574    0.0
581    1.0
591    1.0
603    1.0
613    1.0
621    1.0
631    1.0
639    1.0
644    1.0
649    1.0
657    1.0
659    1.0
668    1.0
673    1.0
680    1.0
Name: computerisable, Length: 67, dtype: float64

##### In this case number of observations are substantially less than the features hence regression will not fit properly
##### Hence in this case we can use feature selection or dimensionality reduction techniques to reduce the number of features 

In [37]:
logisticRegr = LogisticRegression()
logisticRegr.fit(df_trainpca_variable, df_train_target.values.ravel())
predictions_reg = logisticRegr.predict(df_testpca_variable)

In [38]:
a = np.where(df_test_target.values.ravel() > 0.5, 1, 0)
print(a)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [39]:
pd.crosstab(a, predictions_reg, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,166,121,287
1,143,217,360
All,309,338,647


#### I am calculating accuracy based on authors probability output for each role, as we do not have labeled data for whole data set.
#### Accuracy = (162+223)/647 = 59.5%
#### Accuracy Of 1 = 223/348 = 64% (i.e. Predicting computerization of job)

In [40]:
# Using Random Forest to determine the probability of automation similar to author's approach
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit( df_trainpca_variable,df_train_target.values.ravel());

In [41]:
# Use the forest's predict method on the test data of 617 SOC
predictions = rf.predict(df_testpca_variable)
# Calculate the absolute errors
errors = abs(predictions - df_test_target.values.ravel())
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 0.31


#### Mean Absolute Error: 0.31 for random forest model